# Accessing the Lustre (scratch) filesystem on Piz Daint

The file systems can be navigated by double-clicking on the folder names in the File Browser. 
<div class="alert alert-block alert-info">
<b>Note:</b> You can not go up the directory tree from \$HOME (/users/username), so to access \$SCRATCH (/scratch/snx3000/username) you must create a symbolic link.
</div>

In [ ]:
!ln -s $SCRATCH $HOME/scratch

The scratch folder will appear automatically in the file browser after a few seconds.

# Troubleshooting on Piz Daint 
A Jupyter log file is written to `$SCRATCH/jupyterhub_slurmspawner_<jobid>.log`

This should be a first port of call if your notebook is not launching or has stopped working.


# Getting the Python packages you need

The Python environment is built on top of the environment module `cray-python` (currently Python 3.8.5.0)

## Some packages are already available in the Python 3 kernel


### Ipywidgets

In [ ]:
import ipywidgets as widgets
w = widgets.IntSlider()
display(w)

### Matplotlib

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

t = np.arange(0.0, 2.0, 0.01)
s = 1 + np.sin(2*np.pi*t)
plt.plot(t, s)

plt.xlabel('time (s)')
plt.ylabel('voltage (mV)')
plt.title('JupyterLab on Piz Daint')
plt.grid(True)
plt.show()

## Where does Python find packages?

Locations that are searched include:
- `/opt/python/3.8.5.0`
- packages installed along with jupyterlab environment module (paths get added to `$PYTHONPATH`)
- packages installed with pip in your `.local/lib/python3.8/site-packages/`

In [ ]:
pip list

In [ ]:
import sys
for path in sys.path:
    print(path)

In [ ]:
!echo $PYTHONPATH

In [ ]:
!module list

In [ ]:
import numpy as np

In [ ]:
print(np.__version__)

In [ ]:
print(np.__path__)

In [ ]:
import pandas as pd

In [ ]:
print(pd.__version__)

In [ ]:
print(pd.__path__)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import xarray as xr

np.random.seed(123)
xr.set_options(display_style="html")

times = pd.date_range("2000-01-01", "2001-12-31", name="time")
annual_cycle = np.sin(2 * np.pi * (times.dayofyear.values / 365.25 - 0.28))
base = 10 + 15 * annual_cycle.reshape(-1, 1)
tmin_values = base + 3 * np.random.randn(annual_cycle.size, 3)
tmax_values = base + 10 + 3 * np.random.randn(annual_cycle.size, 3)

ds = xr.Dataset(
    {
        "tmin": (("time", "location"), tmin_values),
        "tmax": (("time", "location"), tmax_values),
    },
    {"time": times, "location": ["IA", "IN", "IL"]},
)
ds

## Option 1: Install packages with pip directly
This will install packages in .local/lib/python3.8/site-packages/ 

In [ ]:
pip install seaborn

In [ ]:
pip install xarray

## Option 2: Create a virtual environment and install it as a kernel 
Do this from a fresh login to Daint
```
 $ module load cray-python
 $ module load daint-gpu|mc
 $ module load jupyter-utils
 $ python -m venv [--system-site-packages] venv
 $ source venv/bin/activate
 $ pip install ...
 $ pip install ipykernel
 $ kernel-create -n <name_of_kernel>
 $ deactivate
``` 

Note the difference with and without the `--system-site-packages` option.


Additional pip installs will install into the virtual environment (available in the kernel) which is quite convenient.

## Option 3: Module load environment modules 

You can load modules that CSCS provides, or ones you build and install with EasyBuild.

Place the "module load" commands in a file called `.jupyterhub.env` in your `$HOME`. This file is sourced right before JupyterLab launches.


  

## Option 4: Conda 

if you prefer using conda you can install miniconda and create environments. Turn the environments into kernels using the same kernel-create tool.

```
 $ wget https://repo.anaconda.com/miniconda/Miniconda3-py39_4.9.2-Linux-x86_64.sh
 $ bash Miniconda3-py39_4.9.2-Linux-x86_64.sh
 $ eval "$(/<path-to>/miniconda3/bin/conda shell.bash hook)"
 $ module load daint-gpu
 $ module load jupyter-utils
 $ conda install ipykernel, any other packages you need
 $ kernel-create -n conda_example
```